In [68]:
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file

import fitz # install using: pip install PyMuPDF
from os import listdir
import re, csv

In [2]:
def get_text(pdfpath: str) -> str:
    with fitz.open(pdfpath) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

### 파일 목록 불러오기

In [11]:
# Get problems paths
path = '../data/problems/'
prob_paths = [path+t for t in listdir(path)]
print(prob_paths[:3])

# Get anwsers paths
path2 = '../data/answers/'
ans_paths = [path2+t for t in listdir(path2)]
print(ans_paths[:3])

['../data/problems/230408 국가 9급 건축계획-나.pdf', '../data/problems/230408 국가 9급 건축구조-나.pdf', '../data/problems/230408 국가 9급 경제학개론-나.pdf']
['../data/answers/2023년도 국가공무원 9급 공개경쟁채용 필기시험 최종정답.pdf']


### raw 데이터 추출

In [12]:
# Get texts from problem pdf paths
text_list = list( map(get_text, prob_paths) )
print(text_list[:2])

# // answers
ans_list = list( map(get_text, ans_paths))
print(ans_list)

['2023년도 국가공무원 9급 공채 필기시험\n건축계획\n나 책형\n1쪽\n건축계획\n1. 건축가와 주요 사상 및 대표 작품의 연결이 옳지 않은 것은?\n① 프랭크 로이드 라이트(Frank Lloyd Wright) － 유기적 건축 －\n낙수장(Falling Water)\n② 르 꼬르뷔제(Le Corbusier) － 근대건축의 5원칙 － 라투레트 \n수도원(Sainte Marie de La Tourette)\n③ 미스 반 데어 로에(Mies van der Rohe) － 적을수록 풍부하다\n(Less is more) － 시그램 빌딩(Seagram Building)\n④ 필립 존슨(Philip Johnson) －지역주의－로이드 보험 본사(Lloyd’s \nof London)\n2. 기후대에 따른 토속건축에 대한 설명으로 옳은 것은?\n① 고온건조기후에서는 일사가 충분하므로 이를 최대한 활용하기 \n위해 개구부의 수가 많고 크기 또한 크다.\n② 고온다습기후에서는 증발에 의한 냉각효과가 잘 일어나므로 \n습공기의 실내 체류 시간이 최대한 길게 설계되었다.\n③ 온난기후에서는 따뜻한 기후가 유지되므로 처마 등의 차양으로 \n연중 최대한 일사가 들지 않도록 하였다.\n④ 한랭기후에서는 열손실을 최소로 하는 것이 중요하므로 용적에 \n대한 표면적의 비율이 최소화되었다.\n3. 학교건축계획에서 교과교실형에 대한 설명으로 옳은 것은?\n① 각 학급이 전용 일반교실을 가지며 특정 교과는 특별교실을 두고 \n운영한다.\n② 각 교과의 순수율이 높은 교실이 주어지며 시설의 수준이 높아진다.\n③ 학생의 이동이 적으며 교실 이용률이 100 %라 하더라도 반드시 \n순수율이 높다고 할 수 없다.\n④ 초등학교 저학년에 가장 적합하며 안정적인 생활을 위한 홈베이스가 \n필요하다.\n4. 난방방식에 대한 설명으로 옳지 않은 것은?\n① 온수난방은 난방 휴지기간이 길면 동결의 우려가 있으나 증기난방에 \n비하여 쾌감도는 높다.\n② 증기난방은 현열을 이용하므로 배관 관경이 크고 열의 운반

In [49]:
cleaned_answers = {}

for ans in ans_list:
    title = ans[:ans.find('정답표')].replace('공개경쟁채용', '공채')    # 문제지에서는 '공채' 라고 줄여 쓰더라...
    
    subjects = re.findall('\d+\n[^0-9]*?\n', ans)
    subjects = [i.split('\n')[1] for i in subjects]
    subjects.append('') # for preventing index error
    
    # Cleaning answer data. (removing title and newline for re)
    trash = ans.split(subjects[0])[0][:-2]
    ans = ans.replace(trash, '').replace('\n', '  ') + '  0'  #0 is dumps
    
    cleand_ans = {}
    # Split by subject title
    for i in range(len(subjects)-1):
        q = str('  '+subjects[i]+'\s\s\w[\s\d,]*'+subjects[i+1])
        iter = re.findall(q, ans)
        numbs = [subj.split('  ')[3:-2] for subj in iter][0]
        cleand_ans[re.sub('[^가-힣]*', '', subjects[i])] = numbs    # subject title is only comprised as hangul
    
    cleaned_answers[title.strip()] = cleand_ans

print(len(cleand_ans), cleand_ans)

48 {'건축계획': ['4', '4', '2', '2', '3', '4', '3', '1', '4', '3', '2', '1', '2', '4', '2', '1', '4', '3', '3', '1'], '건축구조': ['2', '4', '1', '4', '2', '3', '4', '3', '1', '1', '3', '2', '1', '3', '2', '1', '4', '4', '3', '4'], '경제학개론': ['3', '4', '4', '1', '2', '3', '4', '2', '4', '3', '3', '4', '1', '4', '1', '3', '1', '3', '2', '4'], '공업화학': ['4', '3', '3', '3', '4', '1', '4', '2', '2', '1', '2', '4', '3', '3', '4', '2', '2', '3', '1', '1'], '공직선거법': ['1', '3', '2', '2', '3', '2', '1', '3', '2', '2', '4', '1', '1', '4', '2', '4', '2', '3', '3', '4'], '관세법개론': ['4', '2', '1', '2', '1', '2', '4', '2', '3', '1', '4', '1', '1', '1', '2', '3', '3', '4', '4', '3'], '교육학개론': ['3', '2', '1', '3', '1', '2', '2', '3', '4', '1', '1', '4', '4', '2', '1', '4', '2', '3', '3', '2'], '교정학개론': ['4', '1', '3', '3', '4', '3', '3', '3', '4', '4', '2', '4', '1', '2', '2', '4', '1', '4', '2', '2'], '국어': ['3', '1', '3', '2', '4', '4', '1', '1', '2', '4', '3', '1', '2', '2', '4', '3', '4', '4', '2', '3'], '국제

In [42]:
def get_question_type(problem:str) -> str:
    if ('설명으로' in problem):     return 'describe'
    if ('모두 고르면' in problem):  return 'choose_all'   
    if ('고르면' in problem):       return 'choose'
    if ('나열' in problem):         return 'list'          # 모델 학습에 적합해보이지 않음.
    if ('연결' in problem):         return 'connect'       # 모델 학습에 적합해보이지 않음.
    if ('빈칸' in problem) \
    or ('(가)' in problem):         return 'blank'         # 모델 학습에 적합해보이지 않음.
    
    return 'none'


def get_correct_wrong(problem:str) -> int:
    if ('옳은' in problem) \
    or ('바르게' in problem): 
        return 1
    
    if ('옳지 않은' in problem) \
    or ('않는' in problem) \
    or ('잘못' in problem):
        return -1
    
    return  0   # 대부분은 긍정 선택지

In [73]:
cleaned_text_list = []

for text in text_list:
    # get answer paper
    where = text.split('\n')[:2]
    
    where[1] = re.sub('[^가-힣]*', '', where[1])   # subject title is only comprised as hangul
    
    # Find question
    text = text.replace('\n', ' ')
    problems = re.findall('\d+\.[\s@+].*?\s*?①', text)
    problems = [i[:-2] for i in problems]   # remove ' \n ①'
    
    
    problems.append(' ')                    # add blank for preventing index error
    
    for i in range(len(problems) -1 ):      # -1 b/c .append(' ')
        
        temp = re.split('\.\s', problems[i])
        (question_number, question_text) = (temp[0], '. '.join(temp[1:]))
        
        choices = text[ text.rfind(problems[i]):text.rfind(problems[i+1]) ].replace(problems[i], '')
        choices = re.split('[①②③④⑤]', choices)[1:]
        
        
        cleaned_text_list.append({
            'where': where[0],
            'subject': where[1],
            'question': question_text,
            'choices': choices,
            'answer': cleaned_answers[where[0]][where[1]][int(question_number)-1], 
            'question_type': get_question_type(question_text),
            'question_positive': get_correct_wrong(question_text),
        })


keys = ['where', 'subject', 'question', 'choices', 'answer', 'question_type', 'question_positive']

with open('./cleand_datasets/2023_9.tsv', 'w', encoding='utf8') as output_file:
    dict_writer = csv.DictWriter(output_file, keys, delimiter='\t')
    dict_writer.writeheader()
    dict_writer.writerows(cleaned_text_list)  